In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import pickle
from matplotlib.backends.backend_pdf import PdfPages
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model, Sequential


from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import Input, UpSampling2D, Flatten, BatchNormalization, Dense, Dropout, GlobalAveragePooling2D


In [ ]:
import scipy
from PIL import Image


In [ ]:
assert tf.test.is_gpu_available()
assert tf.test.is_built_with_cuda()

In [ ]:
def Train_Test_CNNModel(TRAINING_DIR, VALIDATION_DIR, bAugEnable, bDropOutEnable, bTransferLearning, bResNet50):
    IMAGE_DIMENSION      = np.int(224)
    IMAGE_SCALING_DIM    = np.float(256) # np.float(16777216) 
    
    if bTransferLearning == True:
        if bResNet50 == True:
            resnet_model = ResNet50( weights = 'imagenet', include_top=False, input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3)) # weights='imagenet'

            #local_weights_file = 'resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
            #resnet_model.load_weights(local_weights_file)

            for layer in resnet_model.layers:
                if isinstance(layer, BatchNormalization):
                    layer.trainable = True
                else:
                    layer.trainable = False

            model = Sequential()
            model.add(resnet_model)
            model.add(GlobalAveragePooling2D())
            model.add(Dense(1024, activation='relu'))
            model.add(Dropout(.30))
            model.add(BatchNormalization())
            model.add(Dense(1, activation='sigmoid'))

            model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
        else:
            local_weights_file = '/storage/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

            pre_trained_model = InceptionV3(input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3),
                                        include_top=False,
                                        weights=None)

            pre_trained_model.load_weights(local_weights_file)

            for layer in pre_trained_model.layers:
                layer.trainable = False
                print(layer)

            # pre_trained_model.summary()
            last_layer = pre_trained_model.get_layer('mixed7')
            print('last layer output shape: ', last_layer.output_shape)
            last_output = last_layer.output

            # Flatten the output layer to 1 dimension
            x = tf.keras.layers.Flatten()(last_output)
            # Add a fully connected layer with 1,024 hidden units and ReLU activation
            x = tf.keras.layers.Dense(1024, activation='relu')(x)
            # Add a dropout rate of 0.2
            x = tf.keras.layers.Dropout(0.2)(x)
            # Add a final sigmoid layer for classification
            x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

            model = tf.keras.Model(pre_trained_model.input, x)
            model.compile(optimizer=RMSprop(lr=0.0001), loss='binary_crossentropy', metrics=['acc'])
    else:
            # Set the Modelup
            model = tf.keras.models.Sequential([
                tf.keras.layers.Conv2D(32,(3,3),activation = 'relu', input_shape = (IMAGE_DIMENSION,IMAGE_DIMENSION,3)),
                tf.keras.layers.MaxPool2D(2,2),

                tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
                tf.keras.layers.MaxPool2D(2, 2),

                tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
                tf.keras.layers.MaxPool2D(2, 2),

                tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
                tf.keras.layers.MaxPool2D(2, 2),

                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(512, activation = 'relu'),
                tf.keras.layers.Dropout(.2),
                tf.keras.layers.Dense(1, activation='sigmoid')

        ])

            model.compile(optimizer = RMSprop(lr=0.0001), loss = 'binary_crossentropy',metrics=['acc'])

    # Augmentation Enabled or Disabled from Configuration
    if bAugEnable == True:
        train_datagen = ImageDataGenerator(rescale=1.0 / IMAGE_SCALING_DIM,
                                       rotation_range=40,
                                       width_shift_range=0.2,
                                       height_shift_range=0.2,
                                       shear_range=0.2,
                                       zoom_range=0.2,
                                       horizontal_flip=True,
                                       fill_mode='nearest')
    else:
        train_datagen = ImageDataGenerator(rescale=1.0 / IMAGE_SCALING_DIM)

    # TRAIN GENERATOR.
    train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                        batch_size=10,
                                                        class_mode='binary',
                                                        target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION))

    validation_datagen = ImageDataGenerator(rescale=1.0 / IMAGE_SCALING_DIM)  # YOUR CODE HERE

    # NOTE: YOU MUST USE A BATCH SIZE OF 10 (batch_size=10) FOR THE
    # VALIDATION GENERATOR.
    validation_generator = train_datagen.flow_from_directory(VALIDATION_DIR,  # YOUR CODE HERE
                                                             batch_size=10,
                                                             class_mode='binary',
                                                             target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION))


    history = model.fit(
                        train_generator,
                        epochs = 100,
                        verbose = 2,
                        validation_data = validation_generator,
                    )

   # PIK = r"CnnModel1.pkl"
   # with open(PIK, "wb") as f:
   #     pickle.dump(history, f)
   # f.close()

    #model.save("resnet50_catdog.h5")
    #print("Saved model to disk")

    # list all data in history
    print(history.history.keys())
    
    # Plot Trained Model results
    acc     = history.history['acc']
    val_acc = history.history['val_acc']
    loss    = history.history['loss']
    val_loss = history.history['val_loss']

    plt.figure(1, figsize=(12, 10))
    plt.plot(acc, color = 'blue', label='Train')
    plt.plot(val_acc, color = 'red', label='Test')
    plt.grid(b=True, which='major', color='#666666', linestyle='-')
    plt.minorticks_on()
    plt.grid(b=True, which='minor', color='#999999', linestyle='-', alpha=0.2)
    plt.legend()
    plt.title('Accuracy Plot')
    plt.show()
    #pdfobj = PdfPages('Accuracy.pdf')
    #plt.savefig(pdfobj, format='pdf')
    plt.savefig('Accuracy.png')
    #pdfobj.close()
    plt.close()

    plt.figure(2,figsize=(12,10))
    plt.plot( loss ,  color = 'blue', label='Train')
    plt.plot(val_loss,color = 'red', label='Test')
    plt.grid(b=True, which='major', color='#666666', linestyle='-')
    plt.minorticks_on()
    plt.grid(b=True, which='minor', color='#999999', linestyle='-', alpha=0.2)
    plt.legend()
    plt.title('Accuracy Plot')
    plt.savefig('Loss.png')
    plt.close()
    #pdfobj = PdfPages('Loss.pdf')
    #plt.savefig(pdfobj, format='pdf')
    #pdfobj.close()



In [ ]:
TRAINING_DIR = r"datasets/fastai/dogscats/train"
TESTING_DIR = r"datasets/fastai/dogscats/valid"

In [ ]:
bAugEnable          = True
bDropOut            = False
bTransferLearning   = True
bResNet50           = False

Train_Test_CNNModel(TRAINING_DIR,
                    TESTING_DIR,
                    bAugEnable,
                    bDropOut,
                    bTransferLearning,
                    bResNet50)